In [ ]:
#Loading the library 
import pandas as pd
import numpy as np
import os
import os
import math

In [ ]:
# Function using the NRCS-CN method

# Calculate S and CN for 0.2 scenario
# # calculate S 0,2 for the rows 
def calculate_S_02(row):
        QF = row['Quickflow_eckhardt']
        TP = row['total_precipitation_sum']
        return 5*((TP + 2*QF )- math.sqrt(4*math.pow(QF,2) + 5*TP*QF))
    
# calculate C 0,2 for the rows 
def calculate_CN_02(row):
        return 25400/(254+row['S_02'])

# Calculate S and CN for 0.005
# # calculate S 0,05 for the rows 
def calculate_S_005(row):
        QF = row['Quickflow_eckhardt']
        TP = row['total_precipitation_sum']
        return 10*((2*TP + 19*QF )- math.sqrt(361*math.pow(QF,2) + 80*TP*QF))
    
# calculate CN 0,05 for the rows 
def calculate_CN_005(row):
        return 25400/(254+row['S_005'])

# Function using the NRCS-CN method
def quickflow_scs(CN, P, lambda_value):
    """
    Calculate quickflow (runoff) using the NRCS method.
    Parameters:
    - S: Soil retention potential
    - P: Total precipitation

    Returns:
    - Q: Quickflow (runoff)
    """
    S = (25400 / CN) - 254
    Ia = lambda_value * S
    return np.where(P <= Ia, 0, ((P - Ia) ** 2) / (P - Ia + S))

# Function to optimize the NEH Curve Number and calculate runoff metrics
def optimize_CN_NEH(file):
    df = pd.read_csv(file)
    Qo = df['Quickflow_eckhardt'].values
    P = df['total_precipitation_sum'].values
    df['S_02'] = df.apply(calculate_S_02, axis=1)
    df['CN_02'] = df.apply(calculate_CN_02, axis=1)
    df['S_005'] = df.apply(calculate_S_005, axis=1)
    df['CN_005'] = df.apply(calculate_CN_005, axis=1)
    results = {'file': file}
    
    results = {'file': os.path.basename(file)}
    for lambda_value, CN_col in [(0.2, 'CN_02'), (0.05, 'CN_005')]:
        CN_NEH = df[CN_col].NEH()
        Q_estimated = quickflow_scs(CN_NEH, P, lambda_value)
        df[f'Q_estimated_{lambda_value}'] = Q_estimated
        
        # Calculate performance metrics
        mse, rmse, mae = error_metrics(Qo, Q_estimated)
        pbias = PBIAS(Qo, Q_estimated)
        nse = NSE(Qo, Q_estimated)
        kge = KGE(Qo, Q_estimated)
        r2_score = r2(Qo, Q_estimated)
        rse = RSE(Qo, Q_estimated)

        # Store results
        results[f'CN_NEH_{lambda_value}'] = CN_NEH
        results[f'PBIAS_NEH_{lambda_value}'] = pbias
        results[f'NSE_NEH_{lambda_value}'] = nse
        results[f'KGE_NEH_{lambda_value}'] = kge
        results[f'R2_NEH_{lambda_value}'] = r2_score
        results[f'RMSE_NEH_{lambda_value}'] = rmse
        results[f'RSE_NEH_{lambda_value}'] = rse
        
    return df, results

def error_metrics(observed, estimated):
    residuals = observed - estimated
    mse = np.mean(residuals**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(residuals))
    return mse, rmse, mae

def PBIAS(observed, simulated):
    return (np.sum(observed - simulated) / np.sum(observed)) * 100

def NSE(observed, simulated):
    return 1 - (np.sum((observed - simulated) ** 2) / np.sum((observed - np.mean(observed)) ** 2))

def KGE(observed, simulated):
    r = np.corrcoef(observed, simulated)[0, 1]
    alpha = np.std(simulated) / np.std(observed)
    beta = np.mean(simulated) / np.mean(observed)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)

def r2(y_true, y_pred):
    correlation_matrix = np.corrcoef(y_true, y_pred)
    correlation_xy = correlation_matrix[0,1]
    return correlation_xy**2

def RSE(observed, simulated):
    observed = np.array(observed)
    simulated = np.array(simulated)
    n = len(observed)
    
    if n <= 2:
        return np.nan  # Return NaN if we can't compute RSE
    
    # Mean of the observed data
    y_obs_mean = np.mean(observed)
    sum_squared_diff = np.sum((observed - simulated) ** 2)
    sum_squared_total = np.sum((observed - y_obs_mean) ** 2)
    
    # If sum_squared_total is 0, RSE cannot be calculated as it would involve division by zero
    if sum_squared_total == 0:
        return np.nan
    return np.sqrt((n * sum_squared_diff) / ((n - 2) * sum_squared_total))

# Main script execution flow
folder_path = r'\Times series\Group1' # Specify the folder path where the CSV files are located
csv_files = [os.path.join(root, name) for root, dirs, files in os.walk(folder_path) for name in files if name.endswith(".csv")]

results_list = []

for file in csv_files:
    df_modified, results = optimize_CN_NEH(file)
    results_list.append(results)
    modified_file_path = os.path.join(folder_path, f'modified_{os.path.basename(file)}')
    df_modified.to_csv(modified_file_path, index=False)

df_results = pd.DataFrame(results_list)

file_path = r"\Times series\Group1" # Specify the path where the results DataFrame will be saved
df_results.to_csv(os.path.join(file_path,'results_NEH_G1.csv'), index=False)
print(df_results)